## Purpose

This file is used to convert raw data into the master dataset we use for analysis.  The steps taken are as follows:

### Steps:

1. Import Revenue Data - BoxOfficeMojo
2. Import Title Data - IMDB
3. Import Alternate Titles - IMDB
4. Merge Title Data & Alternate Titles - This allows for accurate merging with revenue data.
5. Merge title and revenue data together to create master data set.
6. Save master data set as a '.csv' file.
7. Combine master data with IMDB principal data to create the people_master table.
8. Save the people_master table as a '.csv' file.

All further analysis is based on the '.csv' files created in this notebook.

**1. Import Revenue Data - BoxOfficeMojo**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Import from data source
budget_df = pd.read_csv('data/hollywood_domestic_boxoffice.csv', header=0)

#Update column names to match.  Drop unneeded columns
budget_df.rename(columns = {"title": "primary_title"}, inplace = True)
budget_df.drop(columns = ["rank", "studio"], inplace = True)

# #Add in year of release for accuracy when joining tables.
budget_df['title_year'] = budget_df['primary_title'] + ' - ' + budget_df['year'].astype(str)

**2. Import Title Data - IMDB**

In [ ]:
##need: Primary Title, Genres
##match on: title_year

imdb_titles_df = pd.read_csv('data/imdb.title.basics.csv.gz')

In [ ]:
#Some null values in runtime, genre
imdb_titles_df.info()

**3. Import Alternate Titles - IMDB**

In [ ]:
#Get title akas and merge with imdb titles
imdb_akas_df = pd.read_csv('data/imdb.title.akas.csv.gz')
imdb_akas_df.head()

In [ ]:
#Drop columns.  We only need id and title
imdb_akas_df = imdb_akas_df[['title_id', 'title']]

In [ ]:
#No null values
imdb_akas_df.info()

**4. Merge Title Data with Alternate Titles**

In [ ]:
#Rename column to match
imdb_akas_df.rename(columns = {"title_id": "tconst", "title": "alternate_title"}, inplace = True)

In [ ]:
#Merge title data with alternate titles
titles_with_akas = imdb_akas_df.merge(imdb_titles_df, how='left', on='tconst')

In [ ]:
#Drop any duplicate rows
titles_with_akas[titles_with_akas.duplicated()]
titles_with_akas = titles_with_akas.drop_duplicates()

In [ ]:
#Convert year to string in order to create title_year column
titles_with_akas['start_year'] = titles_with_akas['start_year'].astype(str)
#Combine year and title.  This column allows for more accurate merging.  Eliminates errors with remakes.
titles_with_akas['aka_title_year'] = titles_with_akas['alternate_title'] + ' - ' + titles_with_akas['start_year']

**5. Merge title and revenue data together to create master data set.**

In [ ]:
#Merge with revenue data.
titles_and_rev = titles_with_akas.merge(budget_df, how='left', left_on='aka_title_year', right_on='title_year')

In [ ]:
#No duplicates
titles_and_rev[titles_and_rev.duplicated()]

In [ ]:
#Remove NaN values
no_nas = titles_and_rev.dropna(subset=['lifetime_gross'])

Deal with Errors - Movies Where Alternate Titles == Separate Films

In [ ]:
#Duplicated title ids (tconst)
no_nas.loc[(no_nas.duplicated(subset='tconst') == True)]

In [ ]:
#Locate and remove 5 rows with incorrect data
no_nas = no_nas.reset_index()
no_nas = no_nas.drop([616,616])
no_nas = no_nas.drop([635,635])
no_nas = no_nas.drop([2055,2055])
no_nas = no_nas.drop([2752,2752])
no_nas = no_nas.drop([3057,3057])

#Check that duplicate IDs have been removed
no_nas.loc[(no_nas.duplicated(subset='tconst') == True)]

In [ ]:
#Create Master Table
master_table = no_nas.drop(columns=['index', 'alternate_title', 'primary_title_y', 'primary_title_x', 'aka_title_year', 'year', 'title_year'])

In [ ]:
#Clean & Rename Columns
def clean(col_name):
    cleaned = col_name.title()
    return cleaned
master_table.columns = master_table.columns.map(clean)
master_table = master_table.rename(columns={'Tconst':'Title_ID', 'Original_Title':'Title', 'Start_Year':'Year'})

In [ ]:
#Movies under 60 minutes are not feature-length.  Remove them from the data.
to_drop = master_df.loc[(master_df['Runtime_Minutes'] < 60)]
master_df = master_df.drop(to_drop.index, axis = 0)

In [ ]:
#Check that data is ready to export
master_table.info()

**6. Save master data set as a '.csv' file.**

In [ ]:
master_table.to_csv(path_or_buf="data/Master_Table.csv")

**7. Combine master data with IMDB principal data to create the people_master table.**

In [ ]:
#Read in master table
master_df = master_table

In [ ]:
#Pull in Principal data sets
principals = pd.read_csv('data/imdb.title.principals.csv.gz')
names = pd.read_csv('data/imdb.name.basics.csv.gz')
names_only = names[['nconst', 'primary_name']]

#Merge to get full names in principal data
principals_names = principals.merge(names_only, on='nconst', how='left')
principals_names.drop(columns = ["ordering", "job", "characters"], inplace = True)

#Clean Column Names
principals_names.rename(columns={'tconst':'Title_ID', 'nconst':'Name_ID', 'category':'Role', 'primary_name':'Name'}, inplace = True)

In [ ]:
#Merge Principals with master table based on movie ID
people_master = master_df.merge(principals_names, how='left', on='Title_ID')
people_master = people_master.dropna(subset=['Name'])

**8. Save the people_master table as a '.csv' file.**

In [ ]:
people_master.to_csv(path_or_buf='data/People_Master.csv')